ЗАДАНИЕ 1.
1) Используя либу pymorphy2 определить пол по имени студента из таблицы student для всех студентов
2) Записать пол в отдельную колонку gender таблицы student, значения ("М", "Ж")
3) Скрипт python должен быть умным и сам создать колонку если ее нет
https://pymorphy2.readthedocs.io/en/stable/user/guide.html

ЗАДАНИЕ 2.
1) сделать STORAGE PROCEDURE, которая
а) если нет, то создает таблицу "names" с 4мя полями:
id - автоинкремент
name - varchar(100)
gender - varchar(1), null
len - integer (определить какой точно тип нужен int2, int4, int8 и т.п., еси это поле будет представлять длину строки в поле name),
при условии, что поле "len" - автогенерируемая (вычисляемая) колонка
б) принимает параметрами name, gender
2) сделать скрипт python, который будет принимать те же параметры и вызывать STORAGE PROCEDURE для сохранения в БД.

In [7]:
pip install pymorphy3

   ---------------------------------------- 0.0/53.8 kB ? eta -:--:--
   ------- -------------------------------- 10.2/53.8 kB ? eta -:--:--
   ---------------------- ----------------- 30.7/53.8 kB 330.3 kB/s eta 0:00:01
   -------------------------------------- - 51.2/53.8 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------- 53.8/53.8 kB 348.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.4 MB 1.9 MB/s eta 0:00:05
    --------------------------------------- 0.2/8.4 MB 1.8 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.4 MB 2.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.4 MB 3.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.4 MB 4.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.4/8.4 MB 5.7 MB/s eta 0:00:02
   ------- -------------------------------- 1.5/8.4 MB 5.8 MB/s eta 0:00:02
   -------- ------

In [14]:
import psycopg2


dbname = 'students'
user = 'postgres'
password = 'root'
host = '127.0.0.1'
port = 5432

db_connection_args = dict(
    dbname=dbname, 
    user=user, 
    password=password,
    host=host,
    port=port,
    #options="-c client_encoding=utf8"
)
conn = psycopg2.connect(**db_connection_args)
cur = conn.cursor()
cur.execute("SELECT id, first_name FROM student")
st = cur.fetchall()
st    

[(4, 'Иван'),
 (5, 'Никита'),
 (6, 'Марина'),
 (7, 'Жанна'),
 (8, 'Кирилл'),
 (9, 'Федя'),
 (10, 'Ольга'),
 (11, 'Валя'),
 (12, 'Дмитрий'),
 (13, 'Остап'),
 (14, 'Гена'),
 (15, 'Лена'),
 (16, 'Петр'),
 (17, 'Зина'),
 (18, 'Аскольд'),
 (19, 'Петр'),
 (20, 'Данила'),
 (21, 'Камила'),
 (22, 'Валентин'),
 (23, 'Хасан'),
 (24, 'Ашот'),
 (25, 'Даша'),
 (26, 'Элла'),
 (27, 'Каштан'),
 (28, 'Джон'),
 (29, 'Генрих'),
 (30, 'Арам'),
 (31, 'Ксюша'),
 (32, 'Анна'),
 (33, 'Алина'),
 (3, 'Петр')]

In [ ]:
[
    (id, name, gender),
    (...)
]

In [24]:
import pymorphy3


morph = pymorphy3.MorphAnalyzer()

for s in st:
    fname = s[1]
    parsed_name = morph.parse(fname)
    if not parsed_name:
        continue
    gender = 'М' if  parsed_name[0].tag.gender == 'masc' else 'Ж'
    cur.execute("""
        UPDATE student 
        SET gender=%s 
        WHERE id=%s;
    """, (gender, s[0]))


In [26]:
import pandas as pd
cur.execute("select id, first_name, gender from student")
pd.DataFrame(cur.fetchall())

,0,1,2
0,4,Иван,М
1,5,Никита,М
2,6,Марина,Ж
3,7,Жанна,Ж
4,8,Кирилл,М
5,9,Федя,М
6,10,Ольга,Ж
7,11,Валя,Ж
8,12,Дмитрий,М
9,13,Остап,М


In [14]:
cur.execute("ALTER TABLE student ADD COLUMN IF NOT EXISTS gender VARCHAR(1);")

cur.execute("SELECT * FROM student;")
students = cur.fetchall()

In [16]:
for i, student in enumerate(students):
    gender_value = arr[i]
    cur.execute("""
        UPDATE student 
        SET gender=%s 
        WHERE id=%s;
    """, (gender_value, student[0]))
conn.commit()
cur.close()
conn.close()

In [32]:
dbname='students'
user='postgres'
password='root'
host='127.0.0.1'
port=5432

db_connection_args = dict(
    dbname=dbname, 
    user=user, 
    password=password,
    host=host,
    port=port,
    #options="-c client_encoding=utf8"
)
conn = psycopg2.connect(**db_connection_args)
cur = conn.cursor()
cur.execute("SELECT * FROM student")
st = cur.fetchall()
st 

[(4,
  'Иван',
  'Иванов',
  'Иванович',
  datetime.date(2006, 2, 2),
  525677889912,
  1,
  'М'),
 (5,
  'Никита',
  'Никитин',
  'Олегович',
  datetime.date(2006, 3, 3),
  525677889913,
  1,
  'М'),
 (6,
  'Марина',
  'Маринина',
  'Ивановна',
  datetime.date(2006, 3, 4),
  525677889914,
  1,
  'Ж'),
 (7,
  'Жанна',
  'Сергеева',
  'Алексеевна',
  datetime.date(2006, 1, 1),
  525677889915,
  1,
  'Ж'),
 (8,
  'Кирилл',
  'Сергеев',
  'Кириллович',
  datetime.date(2006, 11, 5),
  525677889916,
  2,
  'М'),
 (9,
  'Федя',
  'Колбаскин',
  'Потапович',
  datetime.date(2006, 10, 10),
  525677889917,
  2,
  'М'),
 (10,
  'Ольга',
  'Петрова',
  'Петровна',
  datetime.date(2006, 5, 6),
  525677889918,
  2,
  'Ж'),
 (11,
  'Валя',
  'Изенштейн',
  'Патрикеевна',
  datetime.date(2006, 12, 3),
  525677889919,
  2,
  'Ж'),
 (12,
  'Дмитрий',
  'Оз',
  'Осипович',
  datetime.date(2006, 6, 7),
  525677889920,
  2,
  'М'),
 (13,
  'Остап',
  'Остапов',
  'Изяславович',
  datetime.date(2006, 2, 22

In [34]:
for s in st:
    name = s[1]
    gender_value = s[7]
    cur.execute("""
        CALL proc(%s, %s);
    """, (name, gender_value))
conn.commit()
cur.close()
conn.close()

1. Прочитать 2 статьи (вдумчиво).
2. Взять текст и использовать его в примерах из 1ой статьи, кроме самого большого кода. 
3. Разбить текст на слова. Удалить стоп-слова. 
4. Загнать текст в пандас. 
5. Добавить колонку лемма.
6. Добавить колонку pos (2ая статья).
7. Загнать дф в постгрес. 
8. Сгруппировать по колонке лемма, посчитать кол-во слов по лемме, вывести максимальное. 
9. Посчитать количество частей речи.

In [16]:
with open('Чичиков.txt', 'r', encoding='utf-8') as f:
    s = f.read()

In [8]:
pip install nltk

In [4]:
import nltk

nltk.download('popular')
nltk.download('wordnet')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users\USER\AppData\Roaming\nltk_dat

True

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
from nltk.tokenize import word_tokenize

text = "NLTK упрощает обработку текста."
word_tokens = word_tokenize(text)
print(word_tokens)

['NLTK', 'упрощает', 'обработку', 'текста', '.']


In [20]:
word_tokens = word_tokenize(s)
print(word_tokens)

['В', 'ворота', 'гостиницы', 'губернского', 'города', 'NN', 'въехала', 'довольно', 'красивая', 'рессорная', 'небольшая', 'бричка', ',', 'в', 'какой', 'ездят', 'холостяки', ':', 'отставные', 'подполковники', ',', 'штабс-капитаны', ',', 'помещики', ',', 'имеющие', 'около', 'сотни', 'душ', 'крестьян', ',', '—', 'словом', ',', 'все', 'те', ',', 'которых', 'называют', 'господами', 'средней', 'руки', '.', 'В', 'бричке', 'сидел', 'господин', ',', 'не', 'красавец', ',', 'но', 'и', 'не', 'дурной', 'наружности', ',', 'ни', 'слишком', 'толст', ',', 'ни', 'слишком', 'тонок', ';', 'нельзя', 'сказать', ',', 'чтобы', 'стар', ',', 'однако', 'ж', 'и', 'не', 'так', 'чтобы', 'слишком', 'молод', '.', 'Въезд', 'его', 'не', 'произвел', 'в', 'городе', 'совершенно', 'никакого', 'шума', 'и', 'не', 'был', 'сопровожден', 'ничем', 'особенным', ';', 'только', 'два', 'русские', 'мужика', ',', 'стоявшие', 'у', 'дверей', 'кабака', 'против', 'гостиницы', ',', 'сделали', 'кое-какие', 'замечания', ',', 'относившиеся', '

In [24]:
from nltk.tokenize import sent_tokenize

text = "OTUS. Наш сайт https://otus.ru/."
sentence_tokens = sent_tokenize(text)
print(sentence_tokens)

['OTUS.', 'Наш сайт https://otus.ru/.']


In [26]:
sentence_tokens = sent_tokenize(s)
print(sentence_tokens)

['В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, в какой ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян, — словом, все те, которых называют господами средней руки.', 'В бричке сидел господин, не красавец, но и не дурной наружности, ни слишком толст, ни слишком тонок; нельзя сказать, чтобы стар, однако ж и не так чтобы слишком молод.', 'Въезд его не произвел в городе совершенно никакого шума и не был сопровожден ничем особенным; только два русские мужика, стоявшие у дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, более к экипажу, чем к сидевшему в нем.', '«Вишь ты, — сказал один другому, — вон какое колесо!', 'что ты думаешь, доедет то колесо, если б случилось, в Москву или не доедет?» — «Доедет», — отвечал другой.', '«А в Казань-то, я думаю, не доедет?» — «В Казань не доедет», — отвечал другой.', 'Этим разговор и кончился.', 'Да еще, когда бричка под

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

text = "NLTK помогает в удалении стоп-слов из текста."
tokens = word_tokenize(text)
stop_words = set(stopwords.words('russian'))
filtered_tokens = [word for word in tokens if word not in stop_words]

print(filtered_tokens)

['NLTK', 'помогает', 'удалении', 'стоп-слов', 'текста', '.']


In [32]:
tokens = word_tokenize(s)
stop_words = set(stopwords.words('russian'))
filtered_tokens = [word for word in tokens if word not in stop_words]

print(filtered_tokens)

['В', 'ворота', 'гостиницы', 'губернского', 'города', 'NN', 'въехала', 'довольно', 'красивая', 'рессорная', 'небольшая', 'бричка', ',', 'ездят', 'холостяки', ':', 'отставные', 'подполковники', ',', 'штабс-капитаны', ',', 'помещики', ',', 'имеющие', 'около', 'сотни', 'душ', 'крестьян', ',', '—', 'словом', ',', 'те', ',', 'которых', 'называют', 'господами', 'средней', 'руки', '.', 'В', 'бричке', 'сидел', 'господин', ',', 'красавец', ',', 'дурной', 'наружности', ',', 'слишком', 'толст', ',', 'слишком', 'тонок', ';', 'сказать', ',', 'стар', ',', 'однако', 'слишком', 'молод', '.', 'Въезд', 'произвел', 'городе', 'совершенно', 'никакого', 'шума', 'сопровожден', 'ничем', 'особенным', ';', 'русские', 'мужика', ',', 'стоявшие', 'дверей', 'кабака', 'против', 'гостиницы', ',', 'сделали', 'кое-какие', 'замечания', ',', 'относившиеся', ',', ',', 'экипажу', ',', 'сидевшему', 'нем', '.', '«', 'Вишь', ',', '—', 'сказал', 'другому', ',', '—', 'вон', 'какое', 'колесо', '!', 'думаешь', ',', 'доедет', 'кол

In [36]:
zn = [',', '.', ';', ':', '!', '?', '«', '»', '—', '-']
filtered_tokens = [word for word in filtered_tokens if word not in zn]
print(filtered_tokens)

['В', 'ворота', 'гостиницы', 'губернского', 'города', 'NN', 'въехала', 'довольно', 'красивая', 'рессорная', 'небольшая', 'бричка', 'ездят', 'холостяки', 'отставные', 'подполковники', 'штабс-капитаны', 'помещики', 'имеющие', 'около', 'сотни', 'душ', 'крестьян', 'словом', 'те', 'которых', 'называют', 'господами', 'средней', 'руки', 'В', 'бричке', 'сидел', 'господин', 'красавец', 'дурной', 'наружности', 'слишком', 'толст', 'слишком', 'тонок', 'сказать', 'стар', 'однако', 'слишком', 'молод', 'Въезд', 'произвел', 'городе', 'совершенно', 'никакого', 'шума', 'сопровожден', 'ничем', 'особенным', 'русские', 'мужика', 'стоявшие', 'дверей', 'кабака', 'против', 'гостиницы', 'сделали', 'кое-какие', 'замечания', 'относившиеся', 'экипажу', 'сидевшему', 'нем', 'Вишь', 'сказал', 'другому', 'вон', 'какое', 'колесо', 'думаешь', 'доедет', 'колесо', 'б', 'случилось', 'Москву', 'доедет', 'Доедет', 'отвечал', 'А', 'Казань-то', 'думаю', 'доедет', 'В', 'Казань', 'доедет', 'отвечал', 'Этим', 'разговор', 'кончил

In [38]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")
text = "Листовые листочки лист листва листве почему так"
tokens = word_tokenize(text)
stemmed_words = [stemmer.stem(word) for word in tokens]
print(stemmed_words)

['листов', 'листочк', 'лист', 'листв', 'листв', 'поч', 'так']


In [46]:
s = ''
for word in filtered_tokens:
    s += word
    s += ' '
s

'В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка ездят холостяки отставные подполковники штабс-капитаны помещики имеющие около сотни душ крестьян словом те которых называют господами средней руки В бричке сидел господин красавец дурной наружности слишком толст слишком тонок сказать стар однако слишком молод Въезд произвел городе совершенно никакого шума сопровожден ничем особенным русские мужика стоявшие дверей кабака против гостиницы сделали кое-какие замечания относившиеся экипажу сидевшему нем Вишь сказал другому вон какое колесо думаешь доедет колесо б случилось Москву доедет Доедет отвечал А Казань-то думаю доедет В Казань доедет отвечал Этим разговор кончился Да бричка подъехала гостинице встретился молодой человек белых канифасовых панталонах весьма узких коротких фраке покушеньями моду из-под которого видна манишка застегнутая тульскою булавкою бронзовым пистолетом Молодой человек оборотился назад посмотрел экипаж придержал рукою ка

In [48]:
stemmer = SnowballStemmer("russian")
tokens = word_tokenize(s)
stemmed_words = [stemmer.stem(word) for word in tokens]
print(stemmed_words)

['в', 'ворот', 'гостиниц', 'губернск', 'город', 'NN', 'въеха', 'довольн', 'красив', 'рессорн', 'небольш', 'бричк', 'езд', 'холостяк', 'отставн', 'подполковник', 'штабс-капита', 'помещик', 'имеющ', 'окол', 'сотн', 'душ', 'крестья', 'слов', 'те', 'котор', 'называ', 'господ', 'средн', 'рук', 'в', 'бричк', 'сидел', 'господин', 'красавец', 'дурн', 'наружн', 'слишк', 'толст', 'слишк', 'тонок', 'сказа', 'стар', 'однак', 'слишк', 'молод', 'въезд', 'произвел', 'город', 'совершен', 'никак', 'шум', 'сопровожд', 'нич', 'особен', 'русск', 'мужик', 'стоя', 'двер', 'кабак', 'прот', 'гостиниц', 'сдела', 'кое-как', 'замечан', 'относ', 'экипаж', 'сидевш', 'нем', 'виш', 'сказа', 'друг', 'вон', 'как', 'колес', 'дума', 'доедет', 'колес', 'б', 'случ', 'москв', 'доедет', 'доедет', 'отвеча', 'а', 'казань-т', 'дума', 'доедет', 'в', 'казан', 'доедет', 'отвеча', 'эт', 'разговор', 'конч', 'да', 'бричк', 'подъеха', 'гостиниц', 'встрет', 'молод', 'человек', 'бел', 'канифасов', 'панталон', 'весьм', 'узк', 'коротк', 

In [50]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [52]:
stemmer = SnowballStemmer("russian")
text = "Лемматизированная форма слова листья это лист"
tokens = word_tokenize(text)
lemmatized_words = [stemmer.stem(word) for word in tokens]
print(lemmatized_words)

['лемматизирова', 'форм', 'слов', 'лист', 'эт', 'лист']


In [54]:
stemmer = SnowballStemmer("russian")
tokens = word_tokenize(s)
lemmatized_words = [stemmer.stem(word) for word in tokens]
print(lemmatized_words)

['в', 'ворот', 'гостиниц', 'губернск', 'город', 'NN', 'въеха', 'довольн', 'красив', 'рессорн', 'небольш', 'бричк', 'езд', 'холостяк', 'отставн', 'подполковник', 'штабс-капита', 'помещик', 'имеющ', 'окол', 'сотн', 'душ', 'крестья', 'слов', 'те', 'котор', 'называ', 'господ', 'средн', 'рук', 'в', 'бричк', 'сидел', 'господин', 'красавец', 'дурн', 'наружн', 'слишк', 'толст', 'слишк', 'тонок', 'сказа', 'стар', 'однак', 'слишк', 'молод', 'въезд', 'произвел', 'город', 'совершен', 'никак', 'шум', 'сопровожд', 'нич', 'особен', 'русск', 'мужик', 'стоя', 'двер', 'кабак', 'прот', 'гостиниц', 'сдела', 'кое-как', 'замечан', 'относ', 'экипаж', 'сидевш', 'нем', 'виш', 'сказа', 'друг', 'вон', 'как', 'колес', 'дума', 'доедет', 'колес', 'б', 'случ', 'москв', 'доедет', 'доедет', 'отвеча', 'а', 'казань-т', 'дума', 'доедет', 'в', 'казан', 'доедет', 'отвеча', 'эт', 'разговор', 'конч', 'да', 'бричк', 'подъеха', 'гостиниц', 'встрет', 'молод', 'человек', 'бел', 'канифасов', 'панталон', 'весьм', 'узк', 'коротк', 

In [58]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [62]:
from nltk.tokenize import word_tokenize
words = word_tokenize(s)
pos = nltk.pos_tag(words)
pos = [p[1] for p in pos]
print(pos)

['JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'JJ', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NN

In [86]:
import pandas as pd
df = pd.DataFrame()
df['Слово'] = filtered_tokens
df['Лемма'] = lemmatized_words
df['Часть_речи'] = pos
df

,Слово,Лемма,Часть_речи
0,В,в,JJ
1,ворота,ворот,NNP
2,гостиницы,гостиниц,NNP
3,губернского,губернск,NNP
4,города,город,NNP
...,...,...,...
2394,узнает,узна,NNP
2395,привело,привел,NNP
2396,совершенное,совершен,NNP
2397,недоумение,недоумен,NNP


In [90]:
import psycopg2
from sqlalchemy import create_engine


dbname='students'
user='postgres'
password='root'
host='127.0.0.1'
port=5432

db_connection_args = dict(
    dbname=dbname, 
    user=user, 
    password=password,
    host=host,
    port=port,
    #options="-c client_encoding=utf8"
)
conn = psycopg2.connect(**db_connection_args)
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

df.to_sql(name='words_test', con=engine, index=False)

conn.close()

SELECT * FROM words_test 
ORDER BY Лемма;
SELECT Лемма, COUNT(Лемма) AS Количество_лемм FROM words_test 
GROUP BY Лемма ORDER BY Количество_лемм DESC LIMIT 1;
SELECT Часть_речи, COUNT(Часть_речи) FROM words_test GROUP BY Часть_речи;